# Project enda : Example B


If you haven't already, read Example A first, it is not long. Download `example_b.zip` and run this notebook in the correct python environment.

Install all the required packages in your python virtualenv:
```bash
pip install numexpr bottleneck pandas enda jupyter h2o scikit-learn statsmodels matplotlib joblib
# more packages used for feature engineering below
pip install jours-feries-france vacances-scolaires-france Unidecode 
```

In this example we will go more in depth, with realistic data and more historical data (~4-5 years). This example is divided in 7 parts:
1. Read and prepare data, check for missing values and gaps
2. Visualize data
3. Feature engineering : datetime and calendar features
4. Portfolio forecast & basic prediction
5. Benchmark with simple evaluation
6. Benchmark with Backtesting 
7. Make the prediction

We set ourselves in a setup as if we were **exactly on 2020-11-30**.
We want to predict the total consumption of customers for the next few days starting 2020-12-01 at a 30min time-step. 
We have:
- our customer contracts until 2020-11-30 included.
- historical load data from 2015-01-01 until 2020-11-15 included. There is a ~15 day time-gap between the last moment for which we have an actual load measure and 'today' (2020-11-30). 
- weather forecast until 2020-12-11 (11 days).
- our TSO's network load forecast until 2020-12-7 (7 days).

In here (example B), we will put all our customers in only 1 group and forecast the next 7 days. We will first construct the dataset and the forecast input data and test it with a basic linear regressor. We will then try various algorithms and compare them. Finally we will give an example of backtesting on the data.  



In [ ]:
import enda
import pandas as pd
import os 

## 1. Read and prepare data, check for missing values and gaps

In [ ]:
# Replace this with the path to your example_b directory.
# You should have ExampleB.ipynb opened in jupiter, so you can run each step 
DIR = '/Users/clement.jeannesson/Jobs/enda_example1/example_b'

In [ ]:
# Get the 30min time-step data just like in Example A 
# (columns are a bit different and there is more data)
# Here we consider all customers in one big group.
def read_data():
    contracts = enda.Contracts.read_contracts_from_file(os.path.join(DIR, "contracts.csv"))
    contracts["contracts_count"] = 1
    portfolio_by_day = enda.Contracts.compute_portfolio_by_day(
        contracts, 
        columns_to_sum = ["contracts_count", "kva"],
        date_start_col="date_start",
        date_end_exclusive_col="date_end_exclusive",
    )
    portfolio = enda.TimeSeries.interpolate_daily_to_sub_daily_data(
        portfolio_by_day, 
        freq='30min', 
        tz='Europe/Paris'
    )
    
    historic_load_measured = pd.read_csv(os.path.join(DIR, "historic_load_measured.csv"))
    weather_and_tso_forecasts = pd.read_csv(os.path.join(DIR, "weather_and_tso_forecasts.csv"))
    # correctly format 'time' as a pandas.DatetimeIndex of dtype: datetime[ns, tzinfo]
    for df in [historic_load_measured, weather_and_tso_forecasts]:
        df['time'] = pd.to_datetime(df['time'])
        df['time'] = enda.TimeSeries.align_timezone(df['time'], tzinfo = 'Europe/Paris')
        df.set_index('time', inplace=True)
     
    # keep only where both loads are known
    historic_load_measured = historic_load_measured.dropna()
    historic_load_measured["load_kw"] = historic_load_measured["smart_metered_kw"] + historic_load_measured["slp_kw"]
    # keep only the full load
    historic_load_measured = historic_load_measured[["load_kw"]]
    
        
    return contracts, portfolio, historic_load_measured, weather_and_tso_forecasts  

In [ ]:
contracts, portfolio, historic_load_measured, weather_and_tso_forecasts = read_data()

In [ ]:
contracts

In [ ]:
portfolio

In [ ]:
historic_load_measured

In [ ]:
# t_weighted is the average french temperature weighted by population density
# t_smooth is a smoothing computed over t_weighted to take into account building calorific inertia 
# (t_smooth is computed out of enda here)

# some tso_forecast_load_mw is missing at the end (we don't show it here)
weather_and_tso_forecasts.dropna(subset=["tso_forecast_load_mw"])

In [ ]:
# lets create the train set with historical data
historic = pd.merge(
    portfolio, 
    historic_load_measured, # here we select only the load of the desired group
    how='inner', left_index=True, right_index=True
)

historic = pd.merge(
    historic, 
    weather_and_tso_forecasts, 
    how='inner', left_index=True, right_index=True
)

In [ ]:
historic

In [ ]:
# check that there is no NaN value
historic.isna().sum()

In [ ]:
# note that the type of the index is precise
historic.index.dtype, type(historic.index)

In [ ]:
# check missing data in the timeseries (based on the time index only)
freq, missing_periods, extra_points = enda.TimeSeries.find_missing_and_extra_periods(
    dti=historic.index, 
    expected_freq = '30min',
    expected_start_datetime = pd.to_datetime('2015-01-01 00:00:00+01:00').astimezone('Europe/Paris'),
    expected_end_datetime = pd.to_datetime('2020-11-30 23:30:00+01:00').astimezone('Europe/Paris')
)
for missing_period in missing_periods:
    print("Missing data from {} to {}.".format(missing_period[0], missing_period[1]))
if len(extra_points) > 0 :
    print("Extra points found: {}".format(extra_points))


We expected the missing data from 2020-11-16 to 2020-11-30, but not from the rest. 

In [ ]:
# Zoom on a daylight savings time change to double-check that it was handled correctly
historic[(historic.index >= '2019-10-27 01:00:00+02:00') & (historic.index < '2019-10-27 03:30:00+01:00')]

## 2. Visualize data 
In order to visualise using pandas, we use the `matplotlib` backend.


In [ ]:
# Show full data set
historic.plot(figsize=(20, 30), subplots=True)

In [ ]:
# Show recent data 
historic[-1000:].plot(figsize=(20, 30), subplots=True)

Don't hesitate to add your own visualisations!

## 3. Feature engineering

Before we train, we will add some features based on the `datetime`, and some calendar features related to national holidays or school holydays.

We use some packages for the holidays, which are used in **enda.feature_engineering.calendar**  : 
```bash
pip install jours-feries-france vacances-scolaires-france Unidecode
```


In [ ]:
import enda.feature_engineering.calendar

In [ ]:
# define the features we want to add before training/predicting
def featurize(df):
    # put datetime features to capture the data frequencies: daily, weekly and yearly periods.  
    df = enda.DatetimeFeature.split_datetime(
        df, split_list = ['minuteofday', 'dayofweek', 'month']
    )
    df = enda.DatetimeFeature.encode_cyclic_datetime_index(
        df, split_list = ['minuteofday', 'dayofweek', 'dayofyear']
    )
    
    # add features about national holidays and school holidays (French holidays here)
    special_days = enda.feature_engineering.calendar.Calendar().get_french_special_days() 
    df = pd.merge(
        df, special_days,
        how='left', left_index=True, right_index=True
    )
    return df
    

In [ ]:
historic 

In [ ]:
historic = enda.DatetimeFeature.split_datetime(
       historic, split_list = ['minuteofday', 'dayofweek', 'month']
    )
historic

In [ ]:
historic = enda.DatetimeFeature.encode_cyclic_datetime_index(
    historic, split_list = ['minuteofday', 'dayofweek', 'dayofyear']
)


In [ ]:
historic

In [ ]:
special_days = enda.feature_engineering.calendar.Calendar().get_french_special_days() 
special_days

In [ ]:
from jours_feries_france import JoursFeries
import unidecode
result = pd.DataFrame()
res = JoursFeries.for_year(2008)
df_res = pd.DataFrame.from_dict(res, orient='index')
df_res.index = df_res.index.map(lambda x: unidecode.unidecode(x))
result = pd.concat([result, df_res.T], ignore_index=True)
result 

In [ ]:
result = pd.DataFrame()
res = JoursFeries.for_year(2008)
df_res = pd.DataFrame.from_dict(res, orient='index')
df_res.index = df_res.index.map(lambda x: unidecode.unidecode(x))
result = pd.concat([result, df_res.T], ignore_index=True)
result 

In [ ]:
historic = pd.merge(
    historic, special_days,
    how='left', left_index=True, right_index=True
)
historic

In [ ]:
#full_train_set = featurize(historic)
full_train_set = historic

In [ ]:
full_train_set

In [ ]:
# train a basic scikit-learn LinearRegression
from enda.ml_backends.sklearn_estimator import EndaSklearnEstimator
from sklearn.linear_model import LinearRegression

lin_reg = EndaSklearnEstimator(LinearRegression())
lin_reg.train(full_train_set, target_col='load_kw')

## 4. Portfolio forecast & basic prediction

We need an estimate of our portfolio in the next few days, the tso_load and weather forecasts. 

In order to get our portfolio in the next few days, here we will just consider the latest trends in our portfolio. 

In another setup, you might want to connect to your sales software or ERP and take into account contracts that will end or start soon.

We will use `enda.Contracts.forecast_portfolio_linear` (which requires the `sklearn` package).

In [ ]:
# we will forecast the portfolio using a linear method
forecast_portfolio = enda.Contracts.forecast_portfolio_linear(
    portfolio_df=portfolio[portfolio.index >= "2020-11-01 00:00:00+02:00"], # # only use recent portfolio trend to forecast the next few days
    start_forecast_date=pd.to_datetime("2020-12-01 00:00:00+01:00").tz_convert("Europe/Paris"),
    end_forecast_date_exclusive=pd.to_datetime("2020-12-08 00:00:00+01:00").tz_convert("Europe/Paris"),
    freq='30min',
    tzinfo='Europe/Paris'
)
forecast_portfolio

In [ ]:
# add weather_and_tso_forecasts
forecast_input_data = pd.merge(
    forecast_portfolio, 
    weather_and_tso_forecasts.dropna(subset=["tso_forecast_load_mw"]), # forecast only where tso is not null for now 
    how='inner', left_index=True, right_index=True
)
# add feature engineering
forecast_input_data = featurize(forecast_input_data)
forecast_input_data

In [ ]:
# show recent portfolio and forecast
for c in ["contracts_count", "kva"]:
    to_plot = pd.merge(
        portfolio[(portfolio.index >= '2020-10-01')][c].to_frame(c+"_historic"),
        forecast_input_data[c].to_frame(c+"_forecast"),
        how='outer', left_index=True, right_index=True
    ) 
    
    to_plot.plot(figsize=(16, 4)) 

In [ ]:
# do the prediction
lin_reg_prediction = lin_reg.predict(forecast_input_data, target_col="load_kw")


In [ ]:
lin_reg_prediction

In [ ]:
# visualize recent load along with our forecast. 
# remember we don't have recent actual load so there is a time-gap. 
to_plot = pd.merge(
    historic["load_kw"][-4000:].to_frame("historic_kw"), 
    lin_reg_prediction.rename(columns={"load_kw": "forecast_kw"}), 
    how='outer', left_index=True, right_index=True
)
to_plot.plot(ylim=0, figsize=(16, 4))     

## 5. Benchmark with simple evaluation

The previous forecast based on linear regression is very limited. Let's try and use a better algorithm ! 

We will define some algorithms using `scikit-klearn` as a machine learning backend and others using `h2o`.

For that we need the `h2o` package:
```bash
pip install h2o
```



In [ ]:
full_train_set

In [ ]:
# here we do a benchmark, we want to compare with actual data, 
# lets say from 2020-11-01 to 2020-11-15
benchmark_train = full_train_set[full_train_set.index < '2020-11-01']
benchmark_test = full_train_set[full_train_set.index >= '2020-11-01']

# save the actual_load in a 'benchmark' dataframe, 
# we will add the predictions of each algo to 'benchmark'
benchmark = benchmark_test["load_kw"].to_frame("actual_load_kw")

benchmark_test = benchmark_test.drop(columns=["load_kw"])
benchmark

In [ ]:
# some parts give ConvergenceWarnings here and we'll ignore them. 
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# use the same method as before to predict a portfolio for 2020-11-01 -> 2020-11-15
benchmark_test_portfolio = forecast_portfolio = enda.Contracts.forecast_portfolio_linear(
    portfolio_df=portfolio[(portfolio.index >= '2020-10-01') & (portfolio.index < '2020-11-01')],
    start_forecast_date=pd.to_datetime("2020-11-01 00:00:00+01:00").tz_convert("Europe/Paris"),
    end_forecast_date_exclusive=pd.to_datetime("2020-11-16 00:00:00+01:00").tz_convert("Europe/Paris"),
    freq='30min',
    tzinfo='Europe/Paris'
)

benchmark_test['kva'] = benchmark_test_portfolio['kva']
benchmark_test['contracts_count'] = benchmark_test_portfolio['contracts_count']
benchmark_test

In [ ]:
# compare portfolio forecast to reality
for c in ["contracts_count", "kva"]:
    to_plot = pd.merge(
        portfolio[(portfolio.index >= '2020-09-01') & (portfolio.index < '2020-11-16')][c].to_frame(c+"_historic"),
        benchmark_test[c].to_frame(c+"_forecast"),
        how='outer', left_index=True, right_index=True
    ) 
    
    to_plot.plot(figsize=(16, 4)) 

Lets define some algorithms then train and predict with them. All the models we define implement the `enda.estimators.EndaEstimator` abstract class (see the docs).

Enda comes with wrappers around scikit-learn and H2O estimators :
- sklearn: `enda.ml_backends.sklearn_estimator.EndaSklearnEstimator`
- H2O: `enda.ml_backends.h2o_estimator.EndaH2OEstimator`


In [ ]:
import time
import h2o
import random
import numpy

from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from enda.ml_backends.h2o_estimator import EndaH2OEstimator  # enda's wrapper around H2O models
from h2o.estimators import H2OGeneralizedLinearEstimator
from h2o.estimators import H2OXGBoostEstimator
from h2o.estimators import H2OGradientBoostingEstimator
from h2o.estimators import H2ORandomForestEstimator
from h2o.estimators import H2ODeepLearningEstimator

In [ ]:
random.seed(17)  # set random seed for reproducibility
numpy.random.seed(17) # for sklearn 
# for h2o we will define it in each model

In [ ]:
all_models = dict()

In [ ]:
# Some models with the sklearn machine learning backend

all_models['sklearn_lin_reg'] = EndaSklearnEstimator(LinearRegression())  

all_models['sklearn_sgd'] = EndaSklearnEstimator(
    Pipeline([('standard_scaler', StandardScaler()),
              ('sgd', SGDRegressor())
             ]
            )
) 

all_models['sklearn_ada_boost'] = EndaSklearnEstimator(AdaBoostRegressor(
    n_estimators=500,
    loss='square',
    learning_rate=0.8)
)  

all_models['sklearn_nn'] = EndaSklearnEstimator(
    Pipeline([('standard_scaler', StandardScaler()),
              ('mlp', MLPRegressor(
                  solver='adam', 
                  activation='relu',
                  hidden_layer_sizes=[48, 48, 24],
                  max_iter=150
              ))
             ]
            )
)

In [ ]:
# Some models with the h2o machine learning backend

all_models['h2o_glm'] = EndaH2OEstimator(H2OGeneralizedLinearEstimator(
    standardize=False, 
    intercept=True,
    seed=17)
)

all_models['h2o_rf'] = EndaH2OEstimator(H2ORandomForestEstimator(
    ntrees=300,
    max_depth=15,
    sample_rate=0.8,
    min_rows=10,
    nbins=52,
    mtries=3,
    seed=17
))

all_models['h2o_gbm'] = EndaH2OEstimator(H2OGradientBoostingEstimator(
    ntrees=500,
    max_depth=5,
    sample_rate=0.5,
    min_rows=5,
    seed=17
))

#all_models['h2o_xgboost'] = EndaH2OEstimator(H2OXGBoostEstimator(
#    **{
#        "ntrees": 500,
#        "max_depth": 5,
#        "sample_rate": 0.8,
#        "min_rows": 10,
#        "seed": 17
#    }
#))

all_models['h2o_nn'] = EndaH2OEstimator(H2ODeepLearningEstimator(
    **{
        "activation": "Tanh",
        "hidden": [48, 48, 24],
        "distribution": "gaussian",
        "epochs": 20,
        "seed": 17
    }
))

In [ ]:
# You can add more models to the benchmark here if you like


In [ ]:
# to train or predict with H2O models, we boot up a local h2o server
h2o.init(nthreads=-1)
h2o.no_progress()

In [ ]:
benchmark_train.iloc[0:10, 2:5]

In [ ]:
benchmark_train.iloc[0:10, 2:5].to_csv("training_test.csv")

In [ ]:
# this should take between 5 and 15 minutes to run (in function of your hardware)
print("Benchmark with {} models : {}\n".format(len(all_models), list(all_models.keys())))
for model_name, model in all_models.items():
    model_start_time = time.time()
    print("Training {} before predicting with it..".format(model_name))
    model.train(benchmark_train, target_col='load_kw')
    model_prediction = model.predict(benchmark_test, target_col='load_kw')
    benchmark[model_name] = model_prediction
    print("{} took {:.1f} seconds.\n".format(model_name, time.time()-model_start_time))


In [ ]:
benchmark

In [ ]:
# visualize predictions
for c in benchmark.columns:
    if c != "actual_load_kw":
        to_plot = benchmark[["actual_load_kw", c]]
        to_plot.plot(ylim=0, figsize=(16, 4)) 

In [ ]:
# compute the mean absolute percentage error of each algo
from enda.scoring import Scoring

In [ ]:
scoring = Scoring(predictions_df=benchmark, target="actual_load_kw")
scoring.mean_absolute_percentage_error().to_frame("mape")

## 6. Benchmark with Backtesting

In traditional machine learning, we need more than just 1 evaluation to test an algorithm. We typically use cross-validation to see if the algorithm is not biased and if it can be expected to work well in most cases. For time-series predictions we cannot do a regular cross-validation because it is not realistic : we always want to train using historical data that happened before the prediction. 

Here we will do **backtesting** week after week. With the given dataset, this means :
- for each week w from early 2019 until the end of the dataset : train using data from the beginning of the dataset (early 2015) until a few days before week w, then eval on w. 
    - the first iteration will train an algorithm using data from 2015 to 2018, then eval on the first week of 2019
    - the second iteration will train using data from 2015 to a bit before the first week of 2019, then eval on the second week of 2019
    - and so on...
- keep the predictions of each time-step using this method, from early 2019 to november 2020.  
- then compare these predictions to the historic data to evaluate the quality of each algorithm. 

This makes most sense if in your production environment, you plan to retrain the algorithm regularly with recent data. 


Backtesting can take a significant amount of time. 
We backtest only 2 linear regressions below in order to have an example that runs fast. Don't hesitate to add other algorithms. 

In [ ]:
all_models = dict()

all_models['sklearn_lin_reg'] = EndaSklearnEstimator(LinearRegression())

all_models['h2o_glm'] = EndaH2OEstimator(H2OGeneralizedLinearEstimator(standardize=False, intercept=True))

In [ ]:
from dateutil.relativedelta import relativedelta
from enda.timezone_utils import TimezoneUtils
portfolio_train_length = relativedelta(months=1)

In [ ]:
historic

In [ ]:
start_backtesting_dt = pd.to_datetime('2019-01-01 00:00:00+01:00').tz_convert('Europe/Paris')
benchmark = historic[historic.index>=start_backtesting_dt]["load_kw"].to_frame("actual_load_kw")
days_in_each_iteration = 28


for model_name, model in all_models.items():

    count_iterations = 0
    
    model_predictions = []

    for train_set, test_set in enda.BackTesting.yield_train_test(
        historic,
        start_eval_datetime=start_backtesting_dt,
        days_between_trains=days_in_each_iteration,
        gap_days_between_train_and_eval=14
    ):
        count_iterations += 1
        if count_iterations <= 2 or count_iterations % 10 == 0:
            print("Model {}, backtesting iteration {}, train set {}->{}, test set {}->{}\n".format(
                   model_name, count_iterations, 
                   train_set.index.min(), train_set.index.max(), 
                   test_set.index.min(), test_set.index.max()))
        
        # featurize
        test_set = test_set.drop(columns=["load_kw"])
        
        # forecast porfolio for the test_set
        pf_train_start = TimezoneUtils.add_interval_to_day_dt(
            day_dt=test_set.index.min(), 
            interval=-portfolio_train_length,
        )
        pf_train = portfolio[(portfolio.index >= pf_train_start) & (portfolio.index < test_set.index.min())]
        
        
        forecast_portfolio = enda.Contracts.forecast_portfolio_linear(
            portfolio_df=pf_train,
            start_forecast_date=test_set.index.min(),
            end_forecast_date_exclusive=test_set.index.max()+relativedelta(minutes=30),
            freq='30min',
            tzinfo='Europe/Paris' 
        )  # recent portfolio trend
        
        test_set['kva'] = forecast_portfolio['kva']
        test_set['contracts_count'] = forecast_portfolio['contracts_count']

        # train and predict
        model.train(train_set, target_col='load_kw')
        model_predictions.append(model.predict(test_set, target_col='load_kw'))
    
    benchmark[model_name] = pd.concat(model_predictions)


In [ ]:
# visualize predictions
for c in benchmark.columns:
    if c != "actual_load_kw":
        to_plot = benchmark[["actual_load_kw", c]]
        to_plot.plot(ylim=0, figsize=(16, 4)) 

In [ ]:
# compute mean absolute percentage error
scoring = Scoring(predictions_df=benchmark, target="actual_load_kw")
scoring.mean_absolute_percentage_error().to_frame("mape")

If you have time/computing power: 
- try more algorithms in the backtesting benchmark, this is longer but more reliable than a simple benchmark (think of it as crossval versus single eval in a non-time-series setup).
- reduce the "days_in_each_iteration" down to 7 if you think you can have a weekly training in your production environment. 

## 7. Make the prediction

Seeing the results from just the basic benchmark, we here decide to predict using h2o's gbm (and our set of hyperparameters). We now need to train it on the full dataset and make the prediction. 

In the input data, the TSO forecast is only available for the next 7 days but the weather forecast is available for the next 11 days. 

We use **EndaEstimatorWithFallback** to be able to predict with or without TSO data.

Checkout more EndaEstimators here: https://github.com/enercoop/enda/blob/main/enda/estimators.py . They work on top of all supported machine learning backends. 




In [ ]:
from enda.estimators import EndaEstimatorWithFallback

In [ ]:
# create the forecast_input_data dataframe

# we will forecast the portfolio for the next 11 days 
forecast_portfolio =  enda.Contracts.forecast_portfolio_linear(
    portfolio_df=portfolio[portfolio.index >= '2020-11-01 00:00:00+01:00'],
    start_forecast_date=pd.to_datetime("2020-12-01 00:00:00+01:00").tz_convert("Europe/Paris"),
    end_forecast_date_exclusive=pd.to_datetime("2020-12-12 00:00:00+01:00").tz_convert("Europe/Paris"),
    freq='30min',
    tzinfo='Europe/Paris'
)

# this time we don't remove rows where tso_forecast is missing
forecast_input_data = pd.merge(
    forecast_portfolio, 
    weather_and_tso_forecasts, 
    how='inner', left_index=True, right_index=True
)
# add feature engineering
forecast_input_data = featurize(forecast_input_data)
forecast_input_data

In [ ]:
# show recent portfolio and forecast
for c in ["contracts_count", "kva"]:
    to_plot = pd.merge(
        portfolio[(portfolio.index >= '2020-10-01')][c].to_frame(c+"_historic"),
        forecast_input_data[c].to_frame(c+"_forecast"),
        how='outer', left_index=True, right_index=True
    ) 
    
    to_plot.plot(figsize=(16, 4)) 

In [ ]:
# tso data is missing after 2020-12-07 :
forecast_input_data[forecast_input_data.index>='2020-12-07 23:00:00+01:00'].head()

In [ ]:
gbm_1 = EndaH2OEstimator(H2OGradientBoostingEstimator(
    ntrees=500,
    max_depth=5,
    sample_rate=0.5,
    min_rows=5
))

gbm_2 = EndaH2OEstimator(H2OGradientBoostingEstimator(
    ntrees=500,
    max_depth=5,
    sample_rate=0.5,
    min_rows=5
))

m = EndaEstimatorWithFallback(
    resilient_column="tso_forecast_load_mw",
    estimator_with=gbm_1,
    estimator_without=gbm_2
)

In [ ]:
m.train(full_train_set, target_col='load_kw')

In [ ]:
import joblib
model_file_path = os.path.join(DIR, "gbm_with_fallback.pickle")

In [ ]:
# save the model for later
joblib.dump(m, filename=model_file_path)

In [ ]:
del m

In [ ]:
# load the model from disk (works even if you shutdown then restarted the H2O server)
m2 = joblib.load(filename=model_file_path)

In [ ]:
m_prediction = m2.predict(forecast_input_data, target_col="load_kw")

In [ ]:
# a good prediction is made until 2020-12-11 
# even where TSO forecast is missing
m_prediction.tail()

In [ ]:
# visualize recent load along with our forecast; remember we don't have recent actual load so there is a time-gap. 
# (remember that the prediction takes weather forecast and more information into account)
to_plot = pd.merge(
    historic["load_kw"][-1000:].to_frame("historic_kw"), 
    m_prediction.rename(columns={"load_kw": "forecast_kw"}), 
    how='outer', left_index=True, right_index=True
)
to_plot.plot(ylim=0, figsize=(16, 4))     

In [ ]:
# don't forget to shutdown your h2o local server
h2o.cluster().shutdown()
# wait for h2o to really finish shutting down
time.sleep(5)

## Conclusion

Thats all for Example B. Check out Example C next. Thanks for reading and don't hesitate to send feeback at: emmanuel.charon@enercoop.org !